In [3]:
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SAMSUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SAMSUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAMSUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import pandas as pd
import dill

In [5]:
def dataCleansing(data):
    with open("preprocessing_function.pkl",'rb') as f:
        cleansing_func = dill.load(f)
        result = cleansing_func(data)
        return result

In [5]:
def preprocess_text(df, col_names=['CATALOG_NM', 'CATALOG_DESC']):
    
    # html태그 확인
    def has_html_tags(col):
        soup = BeautifulSoup(col, 'html.parser')
        if bool(soup.find()) :
            return soup.get_text()
        else:
            return col

    remove_punct_dict = {ord(punct):' ' for punct in string.punctuation}
    lemmar = WordNetLemmatizer()

    # 문장 입력받음 -> 소문자로 변환 -> 구두점 처리 -> 단어로 토큰 -> 불용어 제거 -> 어근 추출
    def len_normalize(text):
        tokens = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
        return [lemmar.lemmatize(token) for token in tokens]


    # 컬럼 선택
    df = df[col_names]

    # DESC 컬럼에 결측치 존재하는 경우
    if df[col_names[1]].isna().sum()!=0:
        df.fillna('',inplace=True)

    # DESC 컬럼 개행문자 제거
    df[col_names[1]] = df[col_names[1]].str.replace('\r',' ').str.replace('\n',' ').str.replace('\\',' ')

    # DESC 컬럼 태그 확인 및 제거
    df[col_names[1]] = df[col_names[1]].apply(has_html_tags)

    # 태그 지우고 공백인 경우, 상품명으로 대체
    mask = df[col_names[1]] == ''  # boolean Series 생성
    df.loc[mask, col_names[1]] = df.loc[mask, col_names[0]]  # 조건을 만족하는 행만 선택하여 업데이트

    # 불완전 태그 제거
    pattern = re.compile(r'<[a-z]\s+[\d\W\s\w]*\s*=?\s?[0-9]*[\d\W\s\w]')
    df[col_names[1]] = df[col_names[1]].apply(lambda x: re.sub(pattern, ' ', x))

    # 불완전 태그 제거한 후 공백인 경우 NM컬럼 값으로 대체
    df[col_names[1]] = df.apply(lambda x: x[col_names[0]] if x[col_names[1]].isspace() else x[col_names[1]], axis=1)

    # 클랜징
    for i in range(len(col_names)):
        df[col_names[i]] = df[col_names[i]].apply(len_normalize)

    return df

In [6]:
data = pd.DataFrame({'CATALOG_NM':["Fresh Carrots"],'CATALOG_DESC':["Product Name: Fresh Carrots Size: 20cm Material: Carrot Color: Red, orange Is product customizable? Yes Price: Curency: USD Price Range: 170-500 Unit: Per metric ton Production time: Quantity: 24001kg Estimated lead time (days): 10days Supply capability: quantity: 2000Metric Ton time: 30days Product classification: Carrot"]})

In [7]:
df = preprocess_text(data)
df

,CATALOG_NM,CATALOG_DESC
0,"[fresh, carrot]","[product, name, fresh, carrot, size, 20cm, mat..."


In [17]:
with open('preprocessing_function_v2.pkl', 'wb') as file:
    dill.dump(preprocess_text, file)

In [20]:
# 함수 로드
with open('preprocessing_function_v2.pkl', 'rb') as file:
    loaded_function = dill.load(file)

In [ ]:
loaded_function(data)

In [24]:
product = ' '.join(df['CATALOG_DESC'][0])

In [27]:
df['CATALOG_DESC'][0]

'product'

In [25]:
product.split()

['product',
 'name',
 'fresh',
 'carrot',
 'size',
 '20cm',
 'material',
 'carrot',
 'color',
 'red',
 'orange',
 'product',
 'customizable',
 'yes',
 'price',
 'curency',
 'usd',
 'price',
 'range',
 '170',
 '500',
 'unit',
 'per',
 'metric',
 'ton',
 'production',
 'time',
 'quantity',
 '24001kg',
 'estimated',
 'lead',
 'time',
 'day',
 '10days',
 'supply',
 'capability',
 'quantity',
 '2000metric',
 'ton',
 'time',
 '30days',
 'product',
 'classification',
 'carrot']

In [11]:
' '.join(df['CATALOG_DESC'])

TypeError: sequence item 0: expected str instance, list found

In [29]:
def plus(a,b):
    return a+b

In [30]:
print(plus(1,3))

4


In [32]:
import pickle

In [33]:
with open("plusFunc.pkl", 'wb') as f:
    pickle.dump(plus, f)

In [43]:
def loaded_func(a,b):
    with open("plusFunc.pkl",'rb') as f:
        return pickle.load(f)(a,b)

In [44]:
loadedFunc(1,3)

4

In [52]:
df['CATALOG_DESC']+df['CATALOG_NM']

0    [product, name, fresh, carrot, size, 20cm, mat...
dtype: object

In [53]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [55]:
pip install tensorflow==20.2.3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==20.2.3 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==20.2.3
You should consider upgrading via the 'c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [57]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [61]:
import h5py

# HDF5 파일 열기
with h5py.File('02-0.805.hdf5', 'r') as f:
    # 파일 내용 확인
    print(list(f.keys()))
    # 가능하다면 메타데이터 확인
    if 'metadata' in f.keys():
        print(f['metadata'])

['model_weights', 'optimizer_weights']
